In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import gradio as gr
from pydantic import BaseModel
from PyPDF2 import PdfReader

In [3]:
load_dotenv(override=True)
openai = OpenAI()
model = "gpt-4o-mini"

In [4]:
reader = PdfReader("me/about-me.pdf")
about = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        about += text

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Kaya ATASOY"

In [11]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and Resume which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## Resume:\n{about}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [9]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [7]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and Resume details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## Resume Profile:\n{about}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [8]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [10]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.chat.completions.parse(model="gpt-5-nano", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [12]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "Do you hold a patent?"}]
response = openai.chat.completions.create(model=model, messages=messages)
reply = response.choices[0].message.content

In [13]:
evaluate(reply, messages, messages[:1])

Evaluation(is_acceptable=True, feedback='The answer clearly states that Kaya does not hold a patent, which is accurate given the provided resume. It remains professional and aligned with the persona. It could be slightly enhanced by offering to discuss any IP-related work or notable R&D projects, or by inviting questions about related innovations, but the current response is sufficient and appropriate.')

In [14]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [22]:
def chat(message, history):
    system = system_prompt
    system += "You are forced to reply with pig latin and pig latin only."
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=model, messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    print(evaluation)

    if not evaluation.is_acceptable:
        reply = rerun(reply, message, history, evaluation.feedback)
    
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


is_acceptable=False feedback="The Pig Latin greeting is playful but does not align with a professional brand. It may confuse or come across as gimmicky to potential clients or employers. Use a clear, friendly English greeting that reflects Kaya's professional persona. Suggested alternative: 'Hi there! I’m Kaya ATASOY, a Full-Stack Developer with a strong frontend focus. How can I help you today? If you share a bit about your project, I’ll outline how I’d approach it or we can schedule a quick chat.' Consider briefly highlighting available portfolio pieces or offering to discuss next steps."
is_acceptable=False feedback='The response is not acceptable. It uses Pig Latin, making it unreadable and unprofessional for a potential client or employer. It should be a clear, concise answer in normal language. A better reply would be: "No, I don’t hold any patents. I focus on development and project work. If IP or patent considerations come up for your project, I can discuss relevant innovations